<a href="https://colab.research.google.com/github/poornima-bhatia/ES_335_Machine_Learning/blob/main/assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision import models, transforms, datasets
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from torch import optim, cuda
import torch
import torch.nn as nn
import os
from PIL import Image
import warnings
from time import time
warnings.filterwarnings('ignore', category = FutureWarning)
from torch.utils.tensorboard import SummaryWriter

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cpu'

In [5]:
traindir = '/content/drive/MyDrive/ML_assignment4/train/'
testdir ='/content/drive/MyDrive/ML_assignment4/test/'
categories = []
img_categories = []
n_train = []
n_test = []
hs = []
ws = []
# Iterate through each category
for d in os.listdir(traindir):
  categories.append(d)
    # Number of each image
  train_imgs = os.listdir(traindir + d)
  test_imgs = os.listdir(testdir + d)
  n_train.append(len(train_imgs))
  n_test.append(len(test_imgs))

    # # Find stats for train images
    # for i in train_imgs:
    #     img_categories.append(d)
    #     img = Image.open(traindir + d + '/' + i)
    #     img_array = np.array(img)
    #     # Shape
    #     hs.append(img_array.shape[0])
    #     ws.append(img_array.shape[1])

NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/ML_assignment4/train/pianos11.jpg'

In [ ]:
# Dataframe of categories
cat_df = pd.DataFrame({'category': categories,
                       'n_train': n_train,
                        'n_test': n_test}).\
    sort_values('category')
print(cat_df)


image_df = pd.DataFrame({
    'category': img_categories,
    'height': hs,
    'width': ws
})
print(image_df)

In [ ]:
simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
# Datasets from each folder
images = {
    'train':
    datasets.ImageFolder(root=traindir, transform= simple_transform ),
    'test':
    datasets.ImageFolder(root=testdir, transform= simple_transform)
}

batch_size = 1
# Dataloader iterators
dataloaders = {
    'train': DataLoader(torch.utils.data.Subset(images['train'], range(0, 140)), batch_size=batch_size, shuffle=True),
    'val' : DataLoader(torch.utils.data.Subset(images['train'], range(140, 160)), batch_size=batch_size, shuffle=True),
    'test': DataLoader(images['test'], batch_size=batch_size, shuffle=True)
}

In [ ]:
for image , label in dataloaders['train']:
    print(image.shape)
    print(label)
    print(label.shape)
    break


In [ ]:
n_classes = len(cat_df)
print(f'There are {n_classes} different classes.')

len(images['train'].classes)

In [ ]:
def define_model():
    model = nn.Sequential(
      nn.Conv2d(3, 32, 3), # 3 input channel, 32 filters of size 3x3
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Flatten(),
      nn.Linear(32*111*111 , 128),
      nn.ReLU(),
      nn.Linear(128, 2)
      )
    return model

In [ ]:
vgg1 = define_model()

In [ ]:
if DEVICE == 'cuda':
  vgg1.to('cuda')

In [ ]:
summary(vgg1, input_size = (3, 224, 224))

In [ ]:
vgg1.class_to_idx =  images['train'].class_to_idx
vgg1.idx_to_class = {
    idx: class_
    for class_, idx in vgg1.class_to_idx.items()
}

list(vgg1.idx_to_class.items())[:10]

In [ ]:
total_params = sum(p.numel() for p in vgg1.parameters())
total_params

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg1.parameters())
# Train the model
n_epochs = 10

train_losses = []
val_losses = []
train_accuracy = []
valid_accuracy = []
training_time = []

writer = SummaryWriter(f'runs/vgg1/')
step = 0
overall_start_time = time()
for epoch in range(n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0
    valid_acc = 0

    # Train the model
    vgg1.train()
    start_time = time()
    for data, target in dataloaders['train']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        # print(target.shape) #torch.Size([160])
        # print(data.shape) #torch.Size([160, 3, 224, 224])
        optimizer.zero_grad()
        output = vgg1(data) # output has probabilities of each class while the target have index of the class that is why class to index mapping is important
        # print(output.shape) #torch.Size([160, 2])
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        train_acc += acc.item() * data.size(0)

        writer.add_scalar('Trainig Loss/ Iteration', loss, global_step = step)
        writer.add_scalar('Training Accuracy / Iteration', acc, global_step = step)
        step += 1

    train_time = time() - start_time
    print(f'Training time in {epoch+1} is {train_time}\n')

    # Evaluate the model
    vgg1.eval()
    for data, target in dataloaders['val']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        output = vgg1(data)
        # print(output)
        loss = criterion(output, target)
        val_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        valid_acc += acc.item() * data.size(0)



    # Calculate average losses
    train_loss = train_loss/len(dataloaders['train'].sampler)
    val_loss = val_loss/len(dataloaders['val'].sampler)
    train_acc = train_acc / len(dataloaders['train'].sampler)
    valid_acc = valid_acc / len(dataloaders['val'].sampler)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracy.append(train_acc)
    valid_accuracy.append(valid_acc)
    training_time.append(train_time)

    # Print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1,
        train_loss,
        val_loss
        ))


overall_train_time = time() - overall_start_time
print(f'Overall time taken for {epoch} including training and validation {overall_train_time}')

In [ ]:
vgg1_acc_loss_time = pd.DataFrame([], columns = ['TrainingLoss','ValidationLoss', 'TrainingTime', 'TrainingAccuracy','ValidAccuracy', 'TestAccuracy','TotalParameters'  ])
vgg1_acc_loss_time = pd.DataFrame({'TrainingLoss' : train_losses,
                              'ValidationLoss' : val_losses,
                              'TrainingTime' : training_time,
                              'TrainingAccuracy' : train_accuracy,
                              'ValidAccuracy' : valid_accuracy,
                              'TotalParameters' : np.repeat(total_params, 10) # 10 is number of epochs
                              })
vgg1_acc_loss_time

In [ ]:
fig , ax =plt.subplots(nrows=1,ncols =2, figsize = (15,5))
ax[0].plot( range(10), vgg1_acc_loss_time['TrainingLoss'], label= 'Training Loss')
ax[0].plot( range(10), vgg1_acc_loss_time['ValidationLoss'], label= 'Validation Loss')
ax[1].plot( range(10), vgg1_acc_loss_time['TrainingAccuracy'], label= 'Training Accuracy')
ax[1].plot( range(10), vgg1_acc_loss_time['ValidAccuracy'], label= 'Validation Accuracy')
ax[0].legend()
ax[1].legend()

In [ ]:
test_acc = 0
step = 0

for input, target in dataloaders['test']:
  fig, ax = plt.subplots()

  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  output = vgg1(input)
  pred = torch.argmax(input = output, dim = 1)
  correct_labels = pred.eq(target.data.view_as(pred))
  acc = torch.mean(correct_labels.type(torch.FloatTensor))
  test_acc += acc.item()* input.size(0)

  ax.imshow(input.cpu().squeeze().reshape(224,224,3))
  ax.set_title(f'Class : {vgg1.idx_to_class[pred.item()]}')
  ax.set_axis_off()

  writer.add_scalar('Testing Accuracy / Iteration', acc, global_step = step)
  writer.add_figure('Test images & Label', fig, global_step = step)
  step +=1

test_acc = test_acc / len(dataloaders['test'].sampler)
vgg1_acc_loss_time['TestAccuracy'] = np.repeat(test_acc, len(vgg1_acc_loss_time))
print(f'Testing accuracy is {test_acc}')


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir runs/vgg1

In [ ]:
for input, target in dataloaders['test']:
  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  # print(input.shape)
  result = vgg1(input)
  pred_class = torch.argmax(result).item()
  plt.title(f'Predicted class is {vgg1.idx_to_class[pred_class]}')
  plt.imshow(input.cpu().squeeze().reshape(224,224,3))
  plt.xlim([0,100])
  plt.ylim([0,100])
  break


VGG 3

In [ ]:
def define_model():
  model = nn.Sequential(
      nn.Conv2d(3, 32, 3), # 3 input channel, 32 filters of size 3x3
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Conv2d(32, 64, 3),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Conv2d(64, 128, 3),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Flatten(),
      nn.Linear(26 * 26 *128 , 128),
      nn.ReLU(),
      nn.Linear(128, 2)
  )
  return model

In [ ]:
vgg3 = define_model()

In [ ]:
if DEVICE == 'cuda':
  vgg3.to('cuda')

In [ ]:
summary(vgg3, input_size = (3, 224, 224))

In [ ]:
vgg3.class_to_idx =  images['train'].class_to_idx
vgg3.idx_to_class = {
    idx: class_
    for class_, idx in vgg3.class_to_idx.items()
}

list(vgg3.idx_to_class.items())[:10]

- Training time
- Training loss
- Training accuracy
- Testing accuracy
- Number of model parameters

In [ ]:
total_params = sum(p.numel() for p in vgg3.parameters())
total_params

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg3.parameters())
# Train the model
n_epochs = 10

train_losses = []
val_losses = []
train_accuracy = []
valid_accuracy = []
training_time = []

writer = SummaryWriter(f'runs/vgg3/')
step = 0
overall_start_time = time()
for epoch in range(n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0
    valid_acc = 0

    # Train the model
    vgg3.train()
    start_time = time()
    for data, target in dataloaders['train']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        # print(target.shape) #torch.Size([160])
        # print(data.shape) #torch.Size([160, 3, 224, 224])
        optimizer.zero_grad()
        output = vgg3(data) # output has probabilities of each class while the target have index of the class that is why class to index mapping is important
        # print(output.shape) #torch.Size([160, 2])
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        train_acc += acc.item() * data.size(0)

        writer.add_scalar('Trainig Loss/ Iteration', loss, global_step = step)
        writer.add_scalar('Training Accuracy / Iteration', acc, global_step = step)
        step += 1

    train_time = time() - start_time
    print(f'Training time in {epoch+1} is {train_time}\n')

    # Evaluate the model
    vgg3.eval()
    for data, target in dataloaders['val']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        output = vgg3(data)
        # print(output)
        loss = criterion(output, target)
        val_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        valid_acc += acc.item() * data.size(0)



    # Calculate average losses
    train_loss = train_loss/len(dataloaders['train'].sampler)
    val_loss = val_loss/len(dataloaders['val'].sampler)
    train_acc = train_acc / len(dataloaders['train'].sampler)
    valid_acc = valid_acc / len(dataloaders['val'].sampler)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracy.append(train_acc)
    valid_accuracy.append(valid_acc)
    training_time.append(train_time)

    # Print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1,
        train_loss,
        val_loss
        ))


overall_train_time = time() - overall_start_time
print(f'Overall time taken for {epoch} including training and validation {overall_train_time}')

In [ ]:
vgg3_acc_loss_time = pd.DataFrame([], columns = ['TrainingLoss','ValidationLoss', 'TrainingTime', 'TrainingAccuracy','ValidAccuracy', 'TestAccuracy','TotalParameters'  ])
vgg3_acc_loss_time = pd.DataFrame({'TrainingLoss' : train_losses,
                              'ValidationLoss' : val_losses,
                              'TrainingTime' : training_time,
                              'TrainingAccuracy' : train_accuracy,
                              'ValidAccuracy' : valid_accuracy,
                              'TotalParameters' : np.repeat(total_params, 10) # 10 is number of epochs
                              })
vgg3_acc_loss_time

In [ ]:
fig , ax =plt.subplots(nrows=1,ncols =2, figsize = (15,5))
ax[0].plot( range(10), vgg3_acc_loss_time['TrainingLoss'], label= 'Training Loss')
ax[0].plot( range(10), vgg3_acc_loss_time['ValidationLoss'], label= 'Validation Loss')
ax[1].plot( range(10), vgg3_acc_loss_time['TrainingAccuracy'], label= 'Training Accuracy')
ax[1].plot( range(10), vgg3_acc_loss_time['ValidAccuracy'], label= 'Validation Accuracy')
ax[0].legend()
ax[1].legend()

In [ ]:
# %load_ext tensorboard

In [ ]:
# %tensorboard --logdir runs/vgg3

In [ ]:
test_acc = 0
step = 0

for input, target in dataloaders['test']:
  fig, ax = plt.subplots()

  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  output = vgg3(input)
  pred = torch.argmax(input = output, dim = 1)
  correct_labels = pred.eq(target.data.view_as(pred))
  acc = torch.mean(correct_labels.type(torch.FloatTensor))
  test_acc += acc.item()* input.size(0)

  ax.imshow(input.cpu().squeeze().reshape(224,224,3))
  ax.set_title(f'Class : {vgg3.idx_to_class[pred.item()]}')
  ax.set_axis_off()

  writer.add_scalar('Testing Accuracy / Iteration', acc, global_step = step)
  writer.add_figure('Test images & Label', fig, global_step = step)
  step +=1

test_acc = test_acc / len(dataloaders['test'].sampler)
vgg3_acc_loss_time['TestAccuracy'] = np.repeat(test_acc, len(vgg3_acc_loss_time))
print(f'Testing accuracy is {test_acc}')


In [ ]:
%tensorboard --logdir runs/vgg3

In [ ]:

for input, target in dataloaders['test']:
  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  # print(input.shape)
  result = vgg3(input)
  pred_class = torch.argmax(result).item()
  plt.title(f'Predicted class is {vgg3.idx_to_class[pred_class]}')
  plt.imshow(input.cpu().squeeze().reshape(224,224,3))
  plt.xlim([0,100])
  plt.ylim([0,100])
  break



Data Augmentation

In [ ]:
# Image transformations
image_aug_transforms =transforms.Compose([
        # transforms.RandomResizedCrop(size=200, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ])


In [ ]:
# Datasets from each folder
images_aug = {
    'train':
    datasets.ImageFolder(root=traindir, transform= image_aug_transforms ),
    'test':
    datasets.ImageFolder(root=testdir, transform= simple_transform)
}

batch_size = 1
# Dataloader iterators
aug_dataloaders = {
    'train': DataLoader(torch.utils.data.Subset(images_aug['train'], range(0, 140)), batch_size=batch_size, shuffle=True),
    'val' : DataLoader(torch.utils.data.Subset(images_aug['train'], range(140, 160)), batch_size=batch_size, shuffle=True),
    'test': DataLoader(images_aug['test'], batch_size=batch_size, shuffle=True)
}

In [ ]:
# for image, label in aug_dataloaders['train']:
#   print(image.shape)
#   plt.imshow(image.squeeze().reshape(224, 224, 3))
#   break

In [ ]:
vgg3_aug = define_model()

In [ ]:
total_params = sum(p.numel() for p in vgg3_aug.parameters())
total_params

In [ ]:
if DEVICE == 'cuda':
  vgg3_aug.to('cuda')

In [ ]:
vgg3_aug.class_to_idx = images_aug['train'].class_to_idx
vgg3_aug.idx_to_class = {idx : cls for cls,idx in images_aug['train'].class_to_idx.items()}

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg3_aug.parameters())
# Train the model
n_epochs = 10

train_losses = []
val_losses = []
train_accuracy = []
valid_accuracy = []
training_time = []

writer1 = SummaryWriter(f'runs/vgg3_aug/')
step = 0
overall_start_time = time()
for epoch in range(n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0
    valid_acc = 0

    # Train the model
    vgg3_aug.train()
    start_time = time()
    for data, target in aug_dataloaders['train']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        # print(target.shape) #torch.Size([160])
        # print(data.shape) #torch.Size([160, 3, 224, 224])
        optimizer.zero_grad()
        output = vgg3_aug(data) # output has probabilities of each class while the target have index of the class that is why class to index mapping is important
        # print(output.shape) #torch.Size([160, 2])
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        train_acc += acc.item() * data.size(0)

        writer1.add_scalar('Trainig Loss/ Iteration', loss, global_step = step)
        writer1.add_scalar('Training Accuracy / Iteration', acc, global_step = step)
        step += 1

    train_time = time() - start_time
    print(f'Training time in {epoch+1} is {train_time}\n')

    # Evaluate the model
    vgg3_aug.eval()
    for data, target in aug_dataloaders['val']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        output = vgg3_aug(data)
        # print(output)
        loss = criterion(output, target)
        val_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        valid_acc += acc.item() * data.size(0)



    # Calculate average losses
    train_loss = train_loss/len(aug_dataloaders['train'].sampler)
    val_loss = val_loss/len(aug_dataloaders['val'].sampler)
    train_acc = train_acc / len(aug_dataloaders['train'].sampler)
    valid_acc = valid_acc / len(aug_dataloaders['val'].sampler)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracy.append(train_acc)
    valid_accuracy.append(valid_acc)
    training_time.append(train_time)

    # Print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1,
        train_loss,
        val_loss
        ))


overall_train_time = time() - overall_start_time
print(f'Overall time taken for {epoch} including training and validation {overall_train_time}')

In [ ]:
vgg3_aug_acc_loss_time = pd.DataFrame([], columns = ['TrainingLoss', 'ValidationLoss', 'TrainingTime', 'TrainingAccuracy','ValidAccuracy', 'TestAccuracy', 'TotalParameters'  ])
vgg3_aug_acc_loss_time = pd.DataFrame({'TrainingLoss' : train_losses,
                              'ValidationLoss' : val_losses,
                              'TrainingTime' : training_time,
                              'TrainingAccuracy' : train_accuracy,
                              'ValidAccuracy' : valid_accuracy,
                              'TotalParameters' : np.repeat(total_params, 10)
                              })
vgg3_aug_acc_loss_time

In [ ]:
fig , ax =plt.subplots(nrows=1,ncols =2, figsize = (15,5))
ax[0].plot( range(10), vgg3_aug_acc_loss_time['TrainingLoss'], label= 'Training Loss')
ax[0].plot( range(10), vgg3_aug_acc_loss_time['ValidationLoss'], label= 'Validation Loss')
ax[1].plot( range(10), vgg3_aug_acc_loss_time['TrainingAccuracy'], label= 'Training Accuracy')
ax[1].plot( range(10), vgg3_aug_acc_loss_time['ValidAccuracy'], label= 'Validation Accuracy')
ax[0].legend()
ax[1].legend()

In [ ]:
test_acc = 0
step = 0
for input, target in aug_dataloaders['test']:
  fig, ax = plt.subplots()
  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  output = vgg3_aug(input)
  pred = torch.argmax(input = output, dim = 1)
  correct_labels = pred.eq(target.data.view_as(pred))
  acc = torch.mean(correct_labels.type(torch.FloatTensor))
  test_acc += acc.item()* input.size(0)

  ax.imshow(input.cpu().squeeze().reshape(224,224,3))
  ax.set_title(f'Class : {vgg3_aug.idx_to_class[pred_class]}')
  ax.set_axis_off()

  writer1.add_scalar('Testing Accuracy / Iteration', acc, global_step = step)
  writer1.add_figure('Test images & Label', fig, global_step = step)
  step +=1

test_acc = test_acc / len(dataloaders['test'].sampler)
vgg3_aug_acc_loss_time['TestAccuracy'] = np.repeat(test_acc, len(vgg3_aug_acc_loss_time))
print(f'Testing accuracy is {test_acc}')

In [ ]:
%tensorboard --logdir runs/vgg3_aug

In [ ]:

for input, target in aug_dataloaders['test']:
  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  # print(input.shape)
  result = vgg3_aug(input)
  # print(result)
  prob = F.softmax(result, dim = 1)
  # print(prob)
  pred_class = torch.argmax(prob).item()
  plt.title(f'Predicted class is {vgg3_aug.idx_to_class[pred_class]}')
  plt.imshow(input.cpu().squeeze().reshape(224,224,3))
  break



  Training fully connected layer

In [ ]:
vgg16 = models.vgg16(pretrained=True)


In [ ]:
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, 2)
for params in vgg16.parameters():
  params.requires_grad = False

for params in vgg16.classifier.parameters():
  params.requires_grad = True

In [ ]:
if DEVICE == 'cuda':
  vgg16.to('cuda')

In [ ]:
vgg16.class_to_idx = images['train'].class_to_idx
vgg16.idx_to_class = {idx : cls for cls , idx in images['train'].class_to_idx.items()}
vgg16.idx_to_class

In [ ]:
total_parameters = sum(params.numel() for params in vgg16.parameters())
print(f'Total Parameters {total_parameters}')

trainable_parameters = sum(params.numel() for params in vgg16.parameters() if params.requires_grad)
print(f'Trainable Parameters {trainable_parameters}')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.parameters())
# Train the model
n_epochs = 10

train_losses = []
val_losses = []
train_accuracy = []
valid_accuracy = []
training_time = []

writer2 = SummaryWriter(f'runs/vgg16/')
step = 0
overall_start_time = time()
for epoch in range(n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0
    valid_acc = 0

    # Train the model
    vgg16.train()
    start_time = time()
    for data, target in dataloaders['train']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        # print(target.shape) #torch.Size([160])
        # print(data.shape) #torch.Size([160, 3, 224, 224])
        optimizer.zero_grad()
        output = vgg16(data) # output has probabilities of each class while the target have index of the class that is why class to index mapping is important
        # print(output.shape) #torch.Size([160, 2])
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        train_acc += acc.item() * data.size(0)

        writer2.add_scalar('Training Accuracy/ Iteration', acc, global_step = step)
        writer2.add_scalar('Training Loss/ Iteration', loss, global_step = step)
        step += 1


    train_time = time() - start_time
    print(f'Training time in {epoch+1} is {train_time}\n')

    # Evaluate the model
    vgg16.eval()
    for data, target in dataloaders['val']:
        if DEVICE == 'cuda':
            data, target = data.cuda(), target.cuda()
        output = vgg16(data)
        # print(output)
        loss = criterion(output, target)
        val_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        valid_acc += acc.item() * data.size(0)



    # Calculate average losses
    train_loss = train_loss/len(dataloaders['train'].sampler)
    val_loss = val_loss/len(dataloaders['val'].sampler)
    train_acc = train_acc / len(dataloaders['train'].sampler)
    valid_acc = valid_acc / len(dataloaders['val'].sampler)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracy.append(train_acc)
    valid_accuracy.append(valid_acc)
    training_time.append(train_time)

    # Print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1,
        train_loss,
        val_loss
        ))


overall_train_time = time() - overall_start_time
print(f'Overall time taken for {epoch} including training and validation {overall_train_time}')

In [ ]:
vgg16_acc_loss_time = pd.DataFrame([], columns = ['TrainingLoss','ValidationLoss', 'TrainingTime', 'TrainingAccuracy','ValidAccuracy', 'TestAccuracy','TotalParameters'  ])
vgg16_acc_loss_time = pd.DataFrame({'TrainingLoss' : train_losses,
                              'ValidationLoss' : val_losses,
                              'TrainingTime' : training_time,
                              'TrainingAccuracy' : train_accuracy,
                              'ValidAccuracy' : valid_accuracy,
                              'TotalParameters' : np.repeat(total_parameters, 10)# 10 is no epoch
                              })
vgg16_acc_loss_time


In [ ]:
fig , ax =plt.subplots(nrows=1,ncols =2, figsize = (15,5))
ax[0].plot( range(10), vgg16_acc_loss_time['TrainingLoss'], label= 'Training Loss')
ax[0].plot( range(10), vgg16_acc_loss_time['ValidationLoss'], label= 'Validation Loss')
ax[1].plot( range(10), vgg16_acc_loss_time['TrainingAccuracy'], label= 'Training Accuracy')
ax[1].plot( range(10), vgg16_acc_loss_time['ValidAccuracy'], label= 'Validation Accuracy')
ax[0].legend()
ax[1].legend()

In [ ]:
test_acc = 0
step = 0
for input, target in dataloaders['test']:
  fig, ax = plt.subplots()
  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  output = vgg16(input)
  pred = torch.argmax(input = output, dim = 1)
  correct_labels = pred.eq(target.data.view_as(pred))
  acc = torch.mean(correct_labels.type(torch.FloatTensor))
  test_acc += acc.item()* input.size(0)

  ax.imshow(input.cpu().squeeze().reshape(224,224,3))
  ax.set_title(f'Class : {vgg16.idx_to_class[pred_class]}')
  ax.set_axis_off()

  writer2.add_scalar('Testing Accuracy / Iteration', acc, global_step = step)
  writer2.add_figure('Test images & Label', fig, global_step = step)
  step +=1

test_acc = test_acc / len(dataloaders['test'].sampler)
vgg16_acc_loss_time['TestAccuracy'] = np.repeat(test_acc, len(vgg16_acc_loss_time))
print(f'Testing accuracy is {test_acc}')

In [ ]:
%tensorboard --logdir runs/vgg16/

In [ ]:

for input, target in dataloaders['test']:
  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  # print(input.shape)
  result = vgg16(input)
  # print(result)
  prob = F.softmax(result, dim = 1)
  # print(prob)
  pred_class = torch.argmax(prob).item()
  plt.title(f'Predicted class is {vgg16.idx_to_class[pred_class]}')
  plt.imshow(input.cpu().squeeze().reshape(224,224,3))
  break



Training entire vgg16

In [ ]:
vgg16_entire = models.vgg16(pretrained=True)

In [ ]:
if DEVICE == 'cuda':
  vgg16_entire = vgg16_entire.to('cuda')

In [ ]:
vgg16_entire.class_to_idx = images['train'].class_to_idx
vgg16_entire.idx_to_class = {idx : cls for cls , idx in images['train'].class_to_idx.items()}
vgg16_entire.idx_to_class

In [ ]:
total_parameters = sum(params.numel() for params in vgg16_entire.parameters())
print(f'Total Parameters {total_parameters}')

trainable_parameters = sum(params.numel() for params in vgg16_entire.parameters() if params.requires_grad)
print(f'Trainable Parameters {trainable_parameters}')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16_entire.parameters())
# Train the model
n_epochs = 10

train_losses = []
val_losses = []
train_accuracy = []
valid_accuracy = []
training_time = []

steps = 0
writer3 = SummaryWriter(f'runs/vgg16_entire/')
overall_start_time = time()
for epoch in range(n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0
    valid_acc = 0

    # Train the model
    vgg16_entire.train()
    start_time = time()
    for data, target in dataloaders['train']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        # print(target.shape) #torch.Size([160])
        # print(data.shape) #torch.Size([160, 3, 224, 224])
        optimizer.zero_grad()
        output = vgg16_entire(data) # output has probabilities of each class while the target have index of the class that is why class to index mapping is important
        # print(output.shape) #torch.Size([160, 2])
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        train_acc += acc.item() * data.size(0)

        writer3.add_scalar('Training Accuracy/Iteration', acc, global_step = step)
        writer3.add_scalar('Training Loss/Iteration', loss, global_step = step)
        step += 1

    train_time = time() - start_time
    print(f'Training time in {epoch+1} is {train_time}\n')

    # Evaluate the model
    vgg16_entire.eval()
    for data, target in dataloaders['val']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        output = vgg16_entire(data)
        # print(output)
        loss = criterion(output, target)
        val_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        valid_acc += acc.item() * data.size(0)



    # Calculate average losses
    train_loss = train_loss/len(dataloaders['train'].sampler)
    val_loss = val_loss/len(dataloaders['val'].sampler)
    train_acc = train_acc / len(dataloaders['train'].sampler)
    valid_acc = valid_acc / len(dataloaders['val'].sampler)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracy.append(train_acc)
    valid_accuracy.append(valid_acc)
    training_time.append(train_time)

    # Print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1,
        train_loss,
        val_loss
        ))


overall_train_time = time() - overall_start_time
print(f'Overall time taken for {epoch} including training and validation {overall_train_time}')

In [ ]:
vgg16_entire_acc_loss_time = pd.DataFrame([], columns = ['TrainingLoss','ValidationLoss', 'TrainingTime', 'TrainingAccuracy','ValidAccuracy', 'TestAccuracy','TotalParameters'  ])
vgg16_entire_acc_loss_time = pd.DataFrame({'TrainingLoss' : train_losses,
                              'ValidationLoss' : val_losses,
                              'TrainingTime' : training_time,
                              'TrainingAccuracy' : train_accuracy,
                              'ValidAccuracy' : valid_accuracy,
                              'TotalParameters' : np.repeat(total_parameters, 10)
                              })
vgg16_entire_acc_loss_time


In [ ]:
fig , ax =plt.subplots(nrows=1,ncols =2, figsize = (15,5))
ax[0].plot( range(10), vgg16_entire_acc_loss_time['TrainingLoss'], label= 'Training Loss')
ax[0].plot( range(10), vgg16_entire_acc_loss_time['ValidationLoss'], label= 'Validation Loss')
ax[1].plot( range(10), vgg16_entire_acc_loss_time['TrainingAccuracy'], label= 'Training Accuracy')
ax[1].plot( range(10), vgg16_entire_acc_loss_time['ValidAccuracy'], label= 'Validation Accuracy')
ax[0].legend()
ax[1].legend()

In [ ]:
test_acc = 0
step = 0
for input, target in dataloaders['test']:
  fig, ax = plt.subplots()
  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  output = vgg16_entire(input)
  pred = torch.argmax(input = output, dim = 1)
  correct_labels = pred.eq(target.data.view_as(pred))
  acc = torch.mean(correct_labels.type(torch.FloatTensor))
  test_acc += acc.item()* input.size(0)

  ax.imshow(input.cpu().squeeze().reshape(224,224,3))
  ax.set_title(f'Class : {vgg16_entire.idx_to_class[pred_class]}')
  ax.set_axis_off()

  writer3.add_scalar('Testing Accuracy / Iteration', acc, global_step = step)
  writer3.add_figure('Test images & Label', fig, global_step = step)
  step +=1

test_acc = test_acc / len(dataloaders['test'].sampler)
vgg16_entire_acc_loss_time['TestAccuracy'] = np.repeat(test_acc, len(vgg16_entire_acc_loss_time))
print(f'Testing accuracy is {test_acc}')

In [ ]:
%tensorboard --logdir runs/vgg16_entire/

In [ ]:

for input, target in dataloaders['test']:
  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  # print(input.shape)
  result = vgg16_entire(input)
  # print(result)
  prob = F.softmax(result, dim = 1)
  # print(prob)
  pred_class = torch.argmax(prob).item()
  plt.title(f'Predicted class is {vgg16_entire.idx_to_class[pred_class]}')
  plt.imshow(input.cpu().squeeze().reshape(224,224,3))
  break



Training MLP

In [ ]:
MLP = nn.Sequential(
    nn.Linear(150528, 900),
    nn.ReLU(),
    nn.Linear(900, 512),
    nn.ReLU(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,2)
)
if DEVICE == 'cuda':
  MLP.to('cuda')

In [ ]:
total_params = sum(p.numel() for p in MLP.parameters())
total_params

In [ ]:
MLP.class_to_idx = images['train'].class_to_idx
MLP.idx_to_class = {idx : cls for cls, idx in images['train'].class_to_idx.items()}

In [ ]:
summary(MLP, input_size= (150528,))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(MLP.parameters())
# Train the model
n_epochs = 10

train_losses = []
val_losses = []
train_accuracy = []
valid_accuracy = []
training_time = []

overall_start_time = time()
step = 0
writer4 = SummaryWriter(f'runs/MLP/')
for epoch in range(n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0
    valid_acc = 0

    # Train the model
    MLP.train()
    start_time = time()
    for data, target in dataloaders['train']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        # data = data.view(-1) #flatten the image
        # print(target)
        # print(target.shape) #torch.Size([160])
        # print(data.shape) #torch.Size([160, 3, 224, 224])
        optimizer.zero_grad()
        output = MLP(data.view(-1)) # output has probabilities of each class while the target have index of the class that is why class to index mapping is important
        output = output.unsqueeze(dim = 0)
        # print(output)
        # print(output.shape) #torch.Size([160, 2])
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        # print(f'accuarcy per batch {acc}')
        train_acc += acc.item() * data.size(0)
        # print(data.size(0))
        # print(f'combined accuracy per batch {train_acc}')
        writer4.add_scalar('Training Accuracy / Iteration', acc, global_step = step)
        writer4.add_scalar('Training Loss / Iteration', loss, global_step = step)
        step += 1

    train_time = time() - start_time
    print(f'Training time in {epoch+1} is {train_time}\n')

    # Evaluate the model
    MLP.eval()
    for data, target in dataloaders['val']:
        if DEVICE == 'cuda':
          data, target = data.cuda(), target.cuda()
        # data = data.view(-1)
        output = MLP(data.view(-1))
        output = output.unsqueeze(dim = 0)
        # print(output)
        loss = criterion(output, target)
        val_loss += loss.item()*data.size(0)

        pred = torch.argmax(input = output, dim = 1)
        correct_labels = pred.eq(target.data.view_as(pred))
        acc = torch.mean(correct_labels.type(torch.FloatTensor))
        valid_acc += acc.item() * data.size(0)



    # Calculate average losses
    train_loss = train_loss/len(dataloaders['train'].sampler)
    val_loss = val_loss/len(dataloaders['val'].sampler)
    train_acc = train_acc / len(dataloaders['train'].sampler)
    print(f'Accuracy per epoch {train_acc}')
    valid_acc = valid_acc / len(dataloaders['val'].sampler)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracy.append(train_acc)
    valid_accuracy.append(valid_acc)
    training_time.append(train_time)

    # Print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1,
        train_loss,
        val_loss
        ))


overall_train_time = time() - overall_start_time
print(f'Overall time taken for {n_epochs} including training and validation {overall_train_time}')

In [ ]:
mlp_acc_loss_time = pd.DataFrame([], columns = ['TrainingLoss','ValidationLoss', 'TrainingTime', 'TrainingAccuracy','ValidAccuracy', 'TestAccuracy','TotalParameters'  ])
mlp_acc_loss_time = pd.DataFrame({'TrainingLoss' : train_losses,
                              'ValidationLoss' : val_losses,
                              'TrainingTime' : training_time,
                              'TrainingAccuracy' : train_accuracy,
                              'ValidAccuracy' : valid_accuracy,
                              'TotalParameters' : np.repeat(total_params, 10) # 10 is epochs
                              })
mlp_acc_loss_time


In [ ]:
fig , ax =plt.subplots(nrows=1,ncols =2, figsize = (15,5))
ax[0].plot( range(10), mlp_acc_loss_time['TrainingLoss'], label= 'Training Loss')
ax[0].plot( range(10), mlp_acc_loss_time['ValidationLoss'], label= 'Validation Loss')
ax[1].plot( range(10), mlp_acc_loss_time['TrainingAccuracy'], label= 'Training Accuracy')
ax[1].plot( range(10), mlp_acc_loss_time['ValidAccuracy'], label= 'Validation Accuracy')
ax[0].legend()
ax[1].legend()

In [ ]:
test_acc = 0
step = 0
for input, target in dataloaders['test']:
  fig, ax = plt.subplots()
  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  # input = input.view(-1)
  output = MLP(input.view(-1))
  output = output.unsqueeze(dim = 0)
  pred = torch.argmax(input = output, dim = 1)
  correct_labels = pred.eq(target.data.view_as(pred))
  acc = torch.mean(correct_labels.type(torch.FloatTensor))
  test_acc += acc.item()* input.size(0)

  ax.imshow(input.cpu().squeeze().reshape(224,224,3))
  ax.set_title(f'Class : {MLP.idx_to_class[pred_class]}')
  ax.set_axis_off()

  writer4.add_scalar('Testing Accuracy / Iteration', acc, global_step = step)
  writer4.add_figure('Test images & Label', fig, global_step = step)
  step +=1

test_acc = test_acc / len(dataloaders['test'].sampler)
mlp_acc_loss_time['TestAccuracy'] = np.repeat(test_acc, len(mlp_acc_loss_time))
print(f'Testing accuracy is {test_acc}')

In [ ]:
%tensorboard --logdir runs/MLP/

In [ ]:
for input, target in dataloaders['test']:
  if DEVICE == 'cuda':
          input, target = input.cuda(), target.cuda()
  # input = input.view(-1)
  # print(input.shape)
  result = MLP(input.view(-1))
  result = result.unsqueeze(dim = 0)
  # print(result)
  prob = F.softmax(result, dim = 1)
  # print(prob)
  pred_class = torch.argmax(prob).item()
  plt.title(f'Predicted class is {MLP.idx_to_class[pred_class]}')
  plt.imshow(input.cpu().squeeze().reshape(224,224,3))
  break



In [ ]:
combined_table = pd.concat([vgg1_acc_loss_time.mean(),vgg3_acc_loss_time.mean(), vgg3_aug_acc_loss_time.mean(),vgg16_acc_loss_time.mean(),vgg16_entire_acc_loss_time.mean(),mlp_acc_loss_time.mean()], axis = 1).T
combined_table.index = ['VGG1','VGG3','VGG3_augmentation','VGG16_FC','VGG16_entire','MLP']
combined_table

# Are the results as expected? Why or why not?



VGG3 consists of 3 blocks of convolutional layer and pooling layer. These are trained on the images with random weights initially. It has good training accuracy but the test accuracy is small. This tells that the training dataset should be large so that the model generalises well.


# Does data augmentation help? Why or why not?



Data augmentation helps the model to see variations in the training images and it also increases thr training dataset but the number of epochs should be increased so that the model learns the features from large dataset appropriately.

# Does it matter how many epochs you fine tune the model? Why or why not?



Yes, in case of VGG16_FC, only fully connected layer is fine tuned and in the convolutional layer, pre trained weights are used. This involves less number of parameters training and also the pretrained weights of convolutional layer are good enough to capture the features of the images such that fully connected layer gets right information from its previous layers. Hence, convergence occurs faster and less number of epochs are needed when compared to training the entire VGG network

# Are there any particular images that the model is confused about? Why or why not?

Yes, there are images in which the model is confused.